<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# Импорт библиотек

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import re
#from scipy import stats as st

from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [2]:
import catboost
import time

from tqdm import tqdm
# from keras.models import Model
# from keras.layers import LSTM, Dense, Masking, Input, Concatenate, Reshape
# from keras.metrics import AUC
# from keras.utils import pad_sequences, plot_model, set_random_seed
# from keras.preprocessing.sequence import pad_sequences
# from keras.optimizers import Adam
# from keras.callbacks import ModelCheckpoint, EarlyStopping
# from keras.regularizers import l1,l2
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix
from catboost import CatBoostClassifier, Pool, cv

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Загрузка файла с данными
total_scores = pd.read_csv('data_01_predict_progress/total_scores.csv')
tournament_scores = pd.read_csv('data_01_predict_progress/tournament_scores.csv')
tournaments = pd.read_csv('data_01_predict_progress/tournaments.csv')
units = pd.read_csv('data_01_predict_progress/units.csv')


In [7]:
units

,id,color,school_id
0,9474,green,244.0
1,733,green,203.0
2,734,green,235.0
3,735,green,168.0
4,736,green,168.0
...,...,...,...
4591,10122,green,198.0
4592,5108,green,62.0
4593,5109,green,27.0
4594,5110,green,62.0


In [25]:
units.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4596 entries, 0 to 4595
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         4596 non-null   int64  
 1   color      4595 non-null   object 
 2   school_id  4007 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 107.8+ KB


In [24]:
units['color'].unique()

array(['green', 'lime', nan], dtype=object)

In [26]:
units['color'].value_counts()

color
green    3800
lime      795
Name: count, dtype: int64

In [29]:
units['school_id'].value_counts()

school_id
62.0     348
198.0    345
111.0    255
38.0     241
192.0    124
        ... 
2.0        1
267.0      1
163.0      1
191.0      1
227.0      1
Name: count, Length: 239, dtype: int64

units
Юнит – спортивная единица. Мы работаем с одиночным катанием, в одном юните только один спортсмен. Если бы были пары/команды, то было б несколько спортсменов в юните.
- id: идентификатор юнита
- color: категория
- school_id: идентификатор школы

In [8]:
tournaments

,id,date_start,date_end,origin_id
0,1,2090-11-29,2090-12-01,2.0
1,2,2091-03-06,2091-03-10,1.0
2,3,2090-10-05,2090-10-08,2.0
3,4,2090-10-18,2090-10-21,2.0
4,5,2090-10-21,2090-10-24,2.0
...,...,...,...,...
137,7114,2092-04-15,2092-04-20,1.0
138,7116,2091-10-24,2091-10-27,0.0
139,7117,2092-04-29,2092-05-03,0.0
140,7115,2092-04-23,2092-04-26,2.0


tournaments
Турнир состоит из нескольких категорий, оценки по категориям расписаны в total_scores
- id: идентификатор турнира
- date_start: дата начала
- date_end: дата завершения
- origin_id: место проведения


In [6]:
total_scores

,id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str
0,442027,304,4785,47.20,43.47,102.70,59.23,0.0,17,1,Короткая программа,x Надбавка за прыжки во второй половине програ...,2,293.74,2
1,442028,604,4785,45.40,46.71,101.19,54.48,0.0,18,2,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,1,294.75,1
2,442029,409,4785,44.00,46.82,99.20,52.38,0.0,6,3,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,4,285.57,4
3,442030,524,4785,46.20,44.86,91.84,47.98,-1.0,3,4,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,3,292.42,3
4,442031,412,4785,44.20,42.59,89.73,48.14,-1.0,15,5,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,6,265.34,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21296,462405,34979,7116,21.67,26.02,46.62,21.10,-0.5,6,20,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за пр...,20,71.66,20
21297,462406,1995,7116,23.20,25.50,46.22,21.22,-0.5,7,21,Произвольная программа,q Прыжок приземлён в четверть e Явно неправиль...,21,71.27,21
21298,462407,34841,7116,21.83,25.00,45.46,20.96,-0.5,5,22,Произвольная программа,q Прыжок приземлён в четверть << Пониженный пр...,22,70.49,22
21299,462408,34980,7116,21.17,26.00,45.24,20.74,-1.5,2,23,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный ...,24,68.22,24


Оценки за выступления по категориям и общие за турнир
- id: идентификатор выступления, джойнится с tournament_scores.total_score_id
- unit_id: идентификатор юнита, ключ к units.id
- tournament_id: идентификатор турнира, tournaments.id
- components_score: артистизм (мастерство, композиция, хореография)
- base_score: базовая оценка за элементы в выступлении (идеал)
- elements_score: реальная оценка всех выполненных элементов, base_score+goe
- decreasings_score: снижения оценок за ошибки
- total_score: components_score+elements_score+decreasings_score за выступление
- starting_place: жеребъёвка
- place: занятое место в категории category_name+segment_name
- segment_name: название сегмента
- info: комментарии и пояснения к оценке
- overall_place: итоговое место в турнире
- overall_total_score: итоговая оценка за весь турнир
- overall_place_str: комментарии, пояснения

In [19]:
display(tournament_scores)

,id,total_score_id,title,decrease,base_score,goe,avg_score
0,1,1,2A,NaN,3.3,0.66,3.96
1,2,1,3F+3Lo,NaN,10.2,-0.11,10.09
2,3,1,3Lz,NaN,0.0,-1.30,5.19
3,4,1,CCoSp4,NaN,3.5,0.56,4.06
4,5,1,FCSp4,NaN,3.2,0.64,3.84
...,...,...,...,...,...,...,...
172153,352008,463407,CCoSp3,NaN,3.0,0.60,3.60
172154,352009,463407,2Lz,NaN,2.1,0.00,2.10
172155,352010,463407,2Lo,NaN,1.7,0.23,1.93
172156,352011,463407,StSq2,NaN,2.6,0.52,3.12


In [37]:
title=tournament_scores['title'].value_counts()

In [106]:
# выполняемые эдементы
title.head(100)

title
CCoSp4        8687
2A            8500
ChSq1         8301
CCoSp3        6064
StSq2         5097
LSp4          4551
3Lo           3904
1A            3379
2Lz           3266
2F            2967
3Lz           2882
FCSp4         2802
FSSp3         2753
StSq1         2725
LSp3          2641
FSSp4         2606
3F            2468
3S            2240
StSqB         2095
StSq3         1943
2S            1914
CCoSp2        1779
2Lo           1776
2A<<          1767
FSSp2         1694
3Lz+3T        1685
FCSp3         1674
1Lz           1644
FCCoSp4       1600
ChSpl1        1588
CSSp4         1578
2A<           1252
2Lz+2Lo       1241
3Lz+2T        1115
LSp2          1058
3S+2T         1018
CSSp3          977
3T             977
2Lz+2T         964
CCoSp1         942
3A             911
SSpB           856
3F+2T          851
3Lo<           830
3Lo+2T         820
2F+2Lo         805
3S<            794
2Lz<           752
FCSp2          750
2A+2T          724
2F<            711
2F!            683
FSSp1 

In [33]:
tournament_scores['decrease'].unique()

array([nan, 'q', '<', '!', '<<', 'nS', 'F', '*', 'e', 'nU', 'nC', '<<*',
       '<*', 'B', 'nS*', '!F', 'F*', '!*', 'q*', 'e*', 'nF', 'nB', 'B.',
       'f*', 'В', '<F', 'qF', 'b', '!<', 'f', '!F*', 'FnU', 'FF'],
      dtype=object)

- '*' - элемент не засчитывается
- q - недокручена четверть
- < - недокручено 90-180 градусов
- << - недокручено больше 180 градусов
- e  - отталкивание не с того ребра
- ! - толчок не с точного ребра
- F - упал
- B - bonus 
- nU
- nC - не было либелы
- nS - не было волчка
- nF
- nB
- v -  можно заменить все ошибки во вращениях

In [56]:
tournament_scores['title'].nunique()

3425

tournament_scores Таблица с оценками поэлементно
- id: идентификатор оценки за конкретный элемент/комбинацию
- total_score_id: идентикатор выступления, ключ total_scores.id
- title: запись элемента или комбинации элементов с отметками об ошибках
- decrease: за что снижена оценка
- base_score: базовая оценка (идеал, цена данного элемента/комбинации, сложность)
- goe: Grade of Execute, качество исполнения, судейские надбавки/убавки
- avg_score: оценка за элемент/комбинацию (усредненная по судьям)

In [57]:
tournament_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172158 entries, 0 to 172157
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              172158 non-null  int64  
 1   total_score_id  172158 non-null  int64  
 2   title           172158 non-null  object 
 3   decrease        41185 non-null   object 
 4   base_score      172158 non-null  float64
 5   goe             172158 non-null  float64
 6   avg_score       172158 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 9.2+ MB


В 'decrease'  много пропусков - это связано с тем, что замечаней по выполнению элементов не было

В "base_score" - присутствуют нули. Исправим это пересчитав значение столбца по формуле: "base_score"= "avg_score" - "goe"

In [58]:
tournament_scores['base_score'] = tournament_scores['avg_score'] - tournament_scores['goe']

In [59]:
tournament_scores.head(20)

,id,total_score_id,title,decrease,base_score,goe,avg_score
0,1,1,2A,NaN,3.30,0.66,3.96
1,2,1,3F+3Lo,NaN,10.20,-0.11,10.09
2,3,1,3Lz,NaN,6.49,-1.30,5.19
3,4,1,CCoSp4,NaN,3.50,0.56,4.06
4,5,1,FCSp4,NaN,3.20,0.64,3.84
5,6,1,LSp4,NaN,2.70,0.49,3.19
6,7,1,StSq2,NaN,2.60,0.26,2.86
7,8,8,2A,NaN,3.30,0.53,3.83
8,9,8,3F,NaN,5.83,0.42,6.25
9,10,8,3Fq+3Loq,q,10.20,-1.91,8.29


In [54]:
figures = ["USp", "LSp", "CSp", "SSp", "FUSp", "FLSp", "FCSp", "FSSp", "CUSp", "CLSp", "CCSp",
                         "CSSp", "CoSp", "CCoSp", "StSq", "ChSq", "2T", "2S", "2Lo", "2F", "2Lz", "2A", "3T", "3S",
                         "3Lo", "3F", "3Lz", "3A", "4T", "4S", "4Lo", "4F", "4Lz", "4A"]

Объединим таблицы: units и total_scores

In [144]:
df = pd.merge(
    units, total_scores,
    left_on='id',
    right_on='unit_id'
)

In [145]:
df.sort_values('id_x', ascending=True)

,id_x,color,school_id,id_y,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str
12334,1,green,198.0,7556,1,33,30.61,22.81,52.67,31.86,0.0,17,13,Короткая программа,< Недокрученный прыжок x Надбавка за прыжки во второй половине программы (10%),15,140.43,NaN
12337,1,green,198.0,10342,1,52,52.46,52.74,95.61,47.87,0.0,6,14,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный прыжок REP Повторение прыжка не в каскаде x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,14,151.55,NaN
12336,1,green,198.0,441807,1,4790,52.39,55.65,111.05,55.40,0.0,28,11,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за прыжки во второй половине программы (10%),10,170.40,10
12335,1,green,198.0,7576,1,33,48.03,43.79,87.76,44.97,0.0,6,15,Произвольная программа,< Недокрученный прыжок << Пониженный прыжок x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,15,140.43,NaN
12333,1,green,198.0,6094,1,21,59.26,55.67,118.09,62.42,0.0,5,4,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный прыжок x Надбавка за прыжки во второй половине программы (10%),4,174.38,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16201,35005,green,302.0,463517,35005,7117,12.54,14.19,26.95,12.76,0.0,1,7,Короткая программа,< Недокрученный прыжок x Надбавка за прыжки во второй половине программы (10%),0,0.00,NaN
16202,35005,green,302.0,463528,35005,7117,20.45,26.50,46.63,20.63,-0.5,5,7,Произвольная программа,< Недокрученный прыжок x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе nS Нет базовой позиции 'волчок',0,0.00,NaN
16430,35006,green,302.0,463619,35006,7117,30.43,38.71,67.63,29.92,-1.0,6,14,Произвольная программа,! Неясное ребро на толчке F/Lz x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,0,0.00,NaN
16429,35006,green,302.0,463615,35006,7117,18.18,19.88,36.13,17.25,-1.0,14,21,Короткая программа,< Недокрученный прыжок x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,0,0.00,NaN


In [146]:
df = df.drop(["id_x"], axis=1)

In [147]:
df = df.rename(columns = {'id_y':'total_score_id'})

In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   color                21301 non-null  object 
 1   school_id            20384 non-null  float64
 2   total_score_id       21301 non-null  int64  
 3   unit_id              21301 non-null  int64  
 4   tournament_id        21301 non-null  int64  
 5   base_score           21301 non-null  float64
 6   components_score     21301 non-null  float64
 7   total_score          21301 non-null  float64
 8   elements_score       21301 non-null  float64
 9   decreasings_score    21301 non-null  float64
 10  starting_place       21301 non-null  int64  
 11  place                21301 non-null  int64  
 12  segment_name         21284 non-null  object 
 13  info                 20720 non-null  object 
 14  overall_place        21301 non-null  int64  
 15  overall_total_score  21284 non-null 

Добавим таблицу tournaments

In [149]:
df = pd.merge(
    df, tournaments,
    left_on='tournament_id',
    right_on='id'
)

In [150]:
pd.set_option('display.max_colwidth', None)

In [151]:

pd.reset_option('display.max_columns')


In [152]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [143]:
df

,color,school_id,unit_id,tournament_id,base_score_x,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,date_start_x,date_end_x,origin_id_x,id_x,total_score_id,title,decrease,base_score_y,goe,avg_score,id_y,date_start_y,date_end_y,origin_id_y
0,green,244.0,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156595,442068,3Lz+3T,NaN,10.10,1.18,11.28,4785,2091-12-20,2091-12-24,2.0
1,green,244.0,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156596,442068,2A,NaN,3.30,0.85,4.15,4785,2091-12-20,2091-12-24,2.0
2,green,244.0,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156597,442068,FCSp4,NaN,3.20,0.87,4.07,4785,2091-12-20,2091-12-24,2.0
3,green,244.0,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156598,442068,3F,NaN,5.83,-1.36,4.47,4785,2091-12-20,2091-12-24,2.0
4,green,244.0,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156599,442068,SSp4,NaN,2.50,0.61,3.11,4785,2091-12-20,2091-12-24,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172153,green,198.0,10122,6885,9.68,17.87,27.85,9.98,0.0,36,5,Произвольная программа.,< Недокрученный прыжок,5,27.85,5,2092-04-06,2092-04-08,0.0,336176,461320,1A,NaN,1.10,0.11,1.21,6885,2092-04-06,2092-04-08,0.0
172154,green,198.0,10122,6885,9.68,17.87,27.85,9.98,0.0,36,5,Произвольная программа.,< Недокрученный прыжок,5,27.85,5,2092-04-06,2092-04-08,0.0,336177,461320,CCoSp1,NaN,2.00,0.27,2.27,6885,2092-04-06,2092-04-08,0.0
172155,green,198.0,10122,6885,9.68,17.87,27.85,9.98,0.0,36,5,Произвольная программа.,< Недокрученный прыжок,5,27.85,5,2092-04-06,2092-04-08,0.0,336178,461320,ChSpl1,NaN,1.50,0.30,1.80,6885,2092-04-06,2092-04-08,0.0
172156,green,198.0,10122,6885,9.68,17.87,27.85,9.98,0.0,36,5,Произвольная программа.,< Недокрученный прыжок,5,27.85,5,2092-04-06,2092-04-08,0.0,336179,461320,2S<,<,1.04,-0.21,0.83,6885,2092-04-06,2092-04-08,0.0


удалим колонки дубликаты "id_x", "id"

In [153]:
df = df.drop(["id"], axis=1)

In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   color                21301 non-null  object 
 1   school_id            20384 non-null  float64
 2   total_score_id       21301 non-null  int64  
 3   unit_id              21301 non-null  int64  
 4   tournament_id        21301 non-null  int64  
 5   base_score           21301 non-null  float64
 6   components_score     21301 non-null  float64
 7   total_score          21301 non-null  float64
 8   elements_score       21301 non-null  float64
 9   decreasings_score    21301 non-null  float64
 10  starting_place       21301 non-null  int64  
 11  place                21301 non-null  int64  
 12  segment_name         21284 non-null  object 
 13  info                 20720 non-null  object 
 14  overall_place        21301 non-null  int64  
 15  overall_total_score  21284 non-null 

Добавим tournament_scores

In [155]:
df = pd.merge(
    df, tournament_scores,
    left_on='total_score_id',
    right_on='total_score_id'
)

In [156]:
df

,color,school_id,total_score_id,unit_id,tournament_id,base_score_x,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,date_start,date_end,origin_id,id,title,decrease,base_score_y,goe,avg_score
0,green,244.0,442068,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156595,3Lz+3T,NaN,10.10,1.18,11.28
1,green,244.0,442068,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156596,2A,NaN,3.30,0.85,4.15
2,green,244.0,442068,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156597,FCSp4,NaN,3.20,0.87,4.07
3,green,244.0,442068,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156598,3F,NaN,5.83,-1.36,4.47
4,green,244.0,442068,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156599,SSp4,NaN,2.50,0.61,3.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172153,green,198.0,461320,10122,6885,9.68,17.87,27.85,9.98,0.0,36,5,Произвольная программа.,< Недокрученный прыжок,5,27.85,5,2092-04-06,2092-04-08,0.0,336176,1A,NaN,1.10,0.11,1.21
172154,green,198.0,461320,10122,6885,9.68,17.87,27.85,9.98,0.0,36,5,Произвольная программа.,< Недокрученный прыжок,5,27.85,5,2092-04-06,2092-04-08,0.0,336177,CCoSp1,NaN,2.00,0.27,2.27
172155,green,198.0,461320,10122,6885,9.68,17.87,27.85,9.98,0.0,36,5,Произвольная программа.,< Недокрученный прыжок,5,27.85,5,2092-04-06,2092-04-08,0.0,336178,ChSpl1,NaN,1.50,0.30,1.80
172156,green,198.0,461320,10122,6885,9.68,17.87,27.85,9.98,0.0,36,5,Произвольная программа.,< Недокрученный прыжок,5,27.85,5,2092-04-06,2092-04-08,0.0,336179,2S<,<,1.04,-0.21,0.83


In [157]:
df = df.drop(["total_score_id"], axis=1)

In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172158 entries, 0 to 172157
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   color                172158 non-null  object 
 1   school_id            164913 non-null  float64
 2   unit_id              172158 non-null  int64  
 3   tournament_id        172158 non-null  int64  
 4   base_score_x         172158 non-null  float64
 5   components_score     172158 non-null  float64
 6   total_score          172158 non-null  float64
 7   elements_score       172158 non-null  float64
 8   decreasings_score    172158 non-null  float64
 9   starting_place       172158 non-null  int64  
 10  place                172158 non-null  int64  
 11  segment_name         172115 non-null  object 
 12  info                 168356 non-null  object 
 13  overall_place        172158 non-null  int64  
 14  overall_total_score  172115 non-null  float64
 15  overall_place_str

In [159]:
df['origin_id'].value_counts()

origin_id
2.0    78204
1.0    55240
0.0    38714
Name: count, dtype: int64

In [161]:
print('Количество турниров:', df['tournament_id'].nunique())

Количество турниров: 142


In [164]:
print('Количество записей по каждому турниру:')
df['tournament_id'].value_counts()

Количество записей по каждому турниру:


tournament_id
6859    4314
78      4223
7114    4045
6848    3660
77      3398
7098    3041
7117    2860
85      2853
7103    2838
79      2786
7110    2769
88      2749
7106    2635
6864    2634
6866    2433
2       2363
24      2255
39      2229
76      2222
68      2218
6699    2203
6885    2011
7092    1982
18      1909
3       1885
14      1873
4992    1863
176     1850
47      1767
40      1728
7111    1724
4790    1709
4793    1638
17      1617
45      1588
48      1550
7109    1547
73      1519
398     1484
6847    1454
84      1430
4789    1412
6845    1410
6160    1403
21      1399
5879    1377
51      1337
177     1296
7115    1281
4794    1266
52      1265
23      1257
4865    1247
33      1245
1       1221
69      1180
91      1160
6850    1159
6849    1155
49      1155
7112    1116
7104    1032
72      1019
222     1009
92      1008
44      1008
263     1001
6676     999
6844     987
43       975
29       960
50       957
2616     948
86       940
22       936
95       93

In [165]:
print('Количество записей по каждому участнику:')
df["unit_id"].value_counts()

Количество записей по каждому участнику:


unit_id
351      442
350      411
354      365
310      363
313      363
        ... 
9526       2
9533       2
9532       2
9521       2
10015      1
Name: count, Length: 3393, dtype: int64

Соберём новую таблицу для изучения данных

In [222]:
new_df = df[['unit_id', 'title', 'decrease', 'date_start', 'date_end' ,'tournament_id']]

In [223]:
new_df

,unit_id,title,decrease,date_start,date_end,tournament_id
0,9474,3Lz+3T,NaN,2091-12-20,2091-12-24,4785
1,9474,2A,NaN,2091-12-20,2091-12-24,4785
2,9474,FCSp4,NaN,2091-12-20,2091-12-24,4785
3,9474,3F,NaN,2091-12-20,2091-12-24,4785
4,9474,SSp4,NaN,2091-12-20,2091-12-24,4785
...,...,...,...,...,...,...
172153,10122,1A,NaN,2092-04-06,2092-04-08,6885
172154,10122,CCoSp1,NaN,2092-04-06,2092-04-08,6885
172155,10122,ChSpl1,NaN,2092-04-06,2092-04-08,6885
172156,10122,2S<,<,2092-04-06,2092-04-08,6885


In [224]:
display(new_df[new_df['unit_id']==9474])

,unit_id,title,decrease,date_start,date_end,tournament_id
0,9474,3Lz+3T,NaN,2091-12-20,2091-12-24,4785
1,9474,2A,NaN,2091-12-20,2091-12-24,4785
2,9474,FCSp4,NaN,2091-12-20,2091-12-24,4785
3,9474,3F,NaN,2091-12-20,2091-12-24,4785
4,9474,SSp4,NaN,2091-12-20,2091-12-24,4785
5,9474,StSq3,NaN,2091-12-20,2091-12-24,4785
6,9474,CCoSp4,NaN,2091-12-20,2091-12-24,4785
7,9474,3Lz,F,2091-12-20,2091-12-24,4785
8,9474,3F<,F,2091-12-20,2091-12-24,4785
9,9474,3Lo,NaN,2091-12-20,2091-12-24,4785


In [225]:
print('Количество турниров для каждого участника')
display(df.groupby('unit_id')['tournament_id'].nunique())

Количество турниров для каждого участника


unit_id
1        15
3        12
4         6
5         6
6        12
         ..
35003     1
35004     1
35005     1
35006     1
35024     1
Name: tournament_id, Length: 3393, dtype: int64

In [568]:
# Функция для извлечения элементов из строки
def extract_elements(text):
    # Паттерн для совпадения элементов, исключая строчную 'e', заглавные 'V' и 'B', и 'q' после 'A', 'T', 'F'
   # pattern = r'\b\d*[A-Z][a-zA-Z]*\d*(?![TFA]q)*(?<![V])\b'
    pattern = r'\b\d*[A-Z][a-zA-Z]*\d*\b'
   # pattern = r'\b\d*[A-Z][a-zA-Z]*\d*V\b'
   # pattern = r'\b\d*[A-Z][a-zA-Z]*(?![VB])\d*(?![VTFA]q)\b'
   # pattern = r'\b\d*[A-Z][a-zA-Z]*(?<![VB])\d*(?![VTFA]q)*(?<![VB])\b'
    # Находим все совпадения
    matches = re.findall(pattern, text)
    # Удаляем 'e' внутри каждого совпадения и обрабатываем случаи с V или B в конце элемента
    final_matches = []
    for match in matches:
        # Убираем 'e'
        match = re.sub(r'e', '', match)
        # Удаляем V или B в конце элемента
        match = re.sub(r'[B]$', '', match)
        match = re.sub(r'[V]$', '', match)
        if re.search(r'[ATFo]q', match):
            # Убираем "q" после "A", "T", "F"
            final_matches.append(re.sub(r'q', '', match))
        else:
            final_matches.append(match)
    return final_matches


# Применение функции к каждому элементу столбца
new_df['list_title'] = new_df['title'].apply(extract_elements)

new_df['list_title'] = new_df['list_title'].apply(lambda x: [item for item in x if item not in ['COMBO', 'REP']])


In [569]:
print(new_df[new_df['title']=='1Eu'])

Empty DataFrame
Columns: [unit_id, title, decrease, date_start, date_end, tournament_id, list_title]
Index: []


In [570]:
new_df.query('unit_id == 1')

,unit_id,title,decrease,date_start,date_end,tournament_id,list_title
100447,1,2A,NaN,2091-12-25,2091-12-26,4844,[2A]
100448,1,3F!,!,2091-12-25,2091-12-26,4844,[3F]
100449,1,LSp4,NaN,2091-12-25,2091-12-26,4844,[LSp4]
100450,1,3Lz+2Lo,NaN,2091-12-25,2091-12-26,4844,"[3Lz, 2Lo]"
100451,1,FCSp4,NaN,2091-12-25,2091-12-26,4844,[FCSp4]
100452,1,StSq2,NaN,2091-12-25,2091-12-26,4844,[StSq2]
100453,1,CCoSp4,NaN,2091-12-25,2091-12-26,4844,[CCoSp4]
100454,1,3T,NaN,2091-12-25,2091-12-26,4844,[3T]
100455,1,3Lz+2T,NaN,2091-12-25,2091-12-26,4844,"[3Lz, 2T]"
100456,1,2A,NaN,2091-12-25,2091-12-26,4844,[2A]


In [564]:
features_df.query('unit_id == 1')

,unit_id,USp,LSp1,LSp2,LSp3,LSp4,CSp1,CSp2,CSp3,CSp4,SSp1,SSp2,SSp3,SSp4,FUSp,FLSp1,FCSp1,FCSp2,FCSp3,FCSp4,FSSp1,FSSp2,FSSp3,FSSp4,CUSp,CLSp,CCSp1,CCSp2,CCSp3,CCSp4,CSSp1,CSSp2,CSSp3,CSSp4,CoSp1,CoSp2,CCoSp1,CCoSp2,CCoSp3,CCoSp4,StSq1,StSq2,StSq3,StSq4,ChSq,1T,2T,3T,4T,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz,1A,2A,3A,4A,1Eu
1980,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,1,0,1,1,0,0,0,1,0,0,1,1,0,0,1,1,0,1,1,1,0,0,1,0,0,1


In [558]:
new_df.head(20)

,unit_id,title,decrease,date_start,date_end,tournament_id,list_title
0,9474,3Lz+3T,NaN,2091-12-20,2091-12-24,4785,"[3Lz, 3T]"
1,9474,2A,NaN,2091-12-20,2091-12-24,4785,[2A]
2,9474,FCSp4,NaN,2091-12-20,2091-12-24,4785,[FCSp4]
3,9474,3F,NaN,2091-12-20,2091-12-24,4785,[3F]
4,9474,SSp4,NaN,2091-12-20,2091-12-24,4785,[SSp4]
5,9474,StSq3,NaN,2091-12-20,2091-12-24,4785,[StSq3]
6,9474,CCoSp4,NaN,2091-12-20,2091-12-24,4785,[CCoSp4]
7,9474,3Lz,F,2091-12-20,2091-12-24,4785,[3Lz]
8,9474,3F<,F,2091-12-20,2091-12-24,4785,[3F]
9,9474,3Lo,NaN,2091-12-20,2091-12-24,4785,[3Lo]


In [545]:
# Список новых фичей
new_features = ["USp", "LSp", "LSp1",'LSp2', 'LSp3', 'LSp4',
                "CSp", "CSp1", "CSp2", "CSp3", "CSp4",
                "SSp", "SSp1",  "SSp2", "SSp3", "SSp4",
                "FUSp",
                "FLSp1", 
                "FCSp", "FCSp1", "FCSp2", "FCSp3", "FCSp4",
                "FSSp", "FSSp1", "FSSp2", "FSSp3", "FSSp4", 
                "CUSp", 
                "CLSp",
                "CCSp", "CCSp1", "CCSp2", "CCSp3", "CCSp4", 
                "CSSp", "CSSp1", "CSSp2", "CSSp3", "CSSp4", 
                "CoSp", "CoSp1", "CoSp2", 
                "CCoSp", "CCoSp1", "CCoSp2", "CCoSp3", "CCoSp4", 
                "StSq", "StSq1", "StSq2", "StSq3", "StSq4", 
                "ChSq", "ChSq1",
                "T", "1T", "2T", "3T", "4T", 
                "S", "1S", "2S", "3S", "4S", 
                "Lo", "1Lo", "2Lo", "3Lo", "4Lo", 
                "F", "1F", "2F", "3F", "4F", 
                "Lz", "1Lz", "2Lz", "3Lz", "4Lz", 
                "A", "1A", "2A", "3A", "4A",
                "1Eu"]

# # Создание таблицы для новых фичей
# features = pd.DataFrame(columns=['unit_id'] + new_features)

# Создание списка для накопления данных
rows = []

# Заполнение таблицы
for unit_id in new_df['unit_id'].unique():
    # Получаем все записи для данного unit_id
    unit_records = new_df[new_df['unit_id'] == unit_id]
    decrease_value = unit_records['decrease'].iloc[0]  # Берем значение decrease из первой записи
    
    # Инициализация словаря для данного unit_id
    unit_dict = {'unit_id': unit_id}
    for feature in new_features:
        # Если 'decrease' равно NaN, устанавливаем значение 1 для всех фичей
        if pd.isna(decrease_value):
            unit_dict[feature] = 1 if any(feature in lst for lst in unit_records['list_title']) else 0
        # Если 'decrease' не равно NaN, устанавливаем значение 0 для всех фичей
        else:
            unit_dict[feature] = 0
    
    # Добавляем словарь в список
    rows.append(unit_dict)

# Преобразование списка словарей в DataFrame
features_df = pd.DataFrame(rows)






# # Заполнение таблицы
# for unit_id in new_df['unit_id'].unique():
#     # Получаем все записи для данного unit_id
#     unit_records = new_df[new_df['unit_id'] == unit_id]
#     decrease_value = unit_records['decrease'].iloc[0]  # Берем значение decrease из первой записи
    
#     # Инициализация словаря для данного unit_id
#     unit_dict = {'unit_id': unit_id}
#     for feature in new_features:
#         # Если 'decrease' равно NaN, устанавливаем значение 1 для всех фичей
#         if pd.isna(decrease_value):
#             unit_dict[feature] = 1 if any(feature in lst for lst in unit_records['list_title']) else 0
#         # Если 'decrease' не равно NaN, устанавливаем значение 0 для всех фичей
#         else:
#             unit_dict[feature] = 0
    
#     # Добавляем значения для данного unit_id в новую таблицу
#     features = features.append(unit_dict, ignore_index=True)


In [546]:
features_df.head(100)

,unit_id,USp,LSp,LSp1,LSp2,LSp3,LSp4,CSp,CSp1,CSp2,CSp3,CSp4,SSp,SSp1,SSp2,SSp3,SSp4,FUSp,FLSp1,FCSp,FCSp1,FCSp2,FCSp3,FCSp4,FSSp,FSSp1,FSSp2,FSSp3,FSSp4,CUSp,CLSp,CCSp,CCSp1,CCSp2,CCSp3,CCSp4,CSSp,CSSp1,CSSp2,CSSp3,CSSp4,CoSp,CoSp1,CoSp2,CCoSp,CCoSp1,CCoSp2,CCoSp3,CCoSp4,StSq,StSq1,StSq2,StSq3,StSq4,ChSq,ChSq1,T,1T,2T,3T,4T,S,1S,2S,3S,4S,Lo,1Lo,2Lo,3Lo,4Lo,F,1F,2F,3F,4F,Lz,1Lz,2Lz,3Lz,4Lz,A,1A,2A,3A,4A,1Eu
0,9474,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1
1,733,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
2,734,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,735,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0
4,736,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,737,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,0,0,1,1,0,0,0,1,0,0,1
6,738,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1
7,739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,740,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0
9,1007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [547]:
#объединим столбцы с одинаковыми по смыслу элементами: 
features_df['LSp1'] = features_df[['LSp', 'LSp1']].max(axis=1)
features_df['CSp1'] = features_df[['CSp', 'CSp1']].max(axis=1) 
features_df['SSp1'] = features_df[['SSp', 'SSp1']].max(axis=1)
features_df['FCSp1'] = features_df[['FCSp', 'FCSp1']].max(axis=1)
features_df['FSSp1'] = features_df[['FSSp', 'FSSp1']].max(axis=1)
features_df['CCSp1'] = features_df[['CCSp', 'CCSp1']].max(axis=1)
features_df['CSSp1'] = features_df[['CSSp', 'CSSp1']].max(axis=1)
features_df['CoSp1'] = features_df[['CoSp', 'CoSp1']].max(axis=1)
features_df['CCoSp1'] =features_df[['CCoSp', 'CCoSp1']].max(axis=1)
features_df['StSq1'] = features_df[['StSq', 'StSq1']].max(axis=1)
features_df['ChSq'] = features_df[['ChSq', 'ChSq1']].max(axis=1)
features_df['1T'] = features_df[['T', '1T']].max(axis=1)
features_df['1S'] = features_df[['S', '1S']].max(axis=1)
features_df['1Lo'] = features_df[['Lo', '1Lo']].max(axis=1)
features_df['1F'] = features_df[['F', '1F']].max(axis=1)
features_df['1Lz'] = features_df[['Lz', '1Lz']].max(axis=1) 
features_df['1A'] = features_df[['A', '1A']].max(axis=1)

In [548]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3393 entries, 0 to 3392
Data columns (total 87 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   unit_id  3393 non-null   int64
 1   USp      3393 non-null   int64
 2   LSp      3393 non-null   int64
 3   LSp1     3393 non-null   int64
 4   LSp2     3393 non-null   int64
 5   LSp3     3393 non-null   int64
 6   LSp4     3393 non-null   int64
 7   CSp      3393 non-null   int64
 8   CSp1     3393 non-null   int64
 9   CSp2     3393 non-null   int64
 10  CSp3     3393 non-null   int64
 11  CSp4     3393 non-null   int64
 12  SSp      3393 non-null   int64
 13  SSp1     3393 non-null   int64
 14  SSp2     3393 non-null   int64
 15  SSp3     3393 non-null   int64
 16  SSp4     3393 non-null   int64
 17  FUSp     3393 non-null   int64
 18  FLSp1    3393 non-null   int64
 19  FCSp     3393 non-null   int64
 20  FCSp1    3393 non-null   int64
 21  FCSp2    3393 non-null   int64
 22  FCSp3    3393 non-null  

In [549]:
features_df = features_df.drop(columns=['LSp', 'CSp', 'SSp', 'FCSp', 'FSSp', 'CCSp', 'CSSp', 'CoSp',
                                       'CCoSp', 'StSq', 'ChSq1', 'T', 'S', 'Lo', 'F', 'Lz', 'A' ])

In [550]:
display(features_df.head())

,unit_id,USp,LSp1,LSp2,LSp3,LSp4,CSp1,CSp2,CSp3,CSp4,SSp1,SSp2,SSp3,SSp4,FUSp,FLSp1,FCSp1,FCSp2,FCSp3,FCSp4,FSSp1,FSSp2,FSSp3,FSSp4,CUSp,CLSp,CCSp1,CCSp2,CCSp3,CCSp4,CSSp1,CSSp2,CSSp3,CSSp4,CoSp1,CoSp2,CCoSp1,CCoSp2,CCoSp3,CCoSp4,StSq1,StSq2,StSq3,StSq4,ChSq,1T,2T,3T,4T,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz,1A,2A,3A,4A,1Eu
0,9474,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,0,1
1,733,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,734,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,735,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0
4,736,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
